In [2]:
import pandas_datareader as pdr
import pandas as pd
import scipy as sp
import numpy as np
from scipy import stats

In [3]:
def GetData(stocks, start_date,end_date):
    #Assigns pandas dataframe and the average rfr as global variable
    global E_rf, data
    #Retrieves adjusted clsoe data for given securities and converts to log returns
    data = pdr.DataReader(stocks, data_source='yahoo', start = start_date, end = end_date)['Adj Close']
    data = (np.log(data) - np.log(data.shift(1))).dropna()
    
    #Retrieves daily yield for 5-year Tbill, takes the average and converts to a decimal percantage to match rest of data
    E_rf = pdr.DataReader('^FVX', data_source='yahoo', start = start_date, end = end_date)['Adj Close']
    E_rf = np.mean(E_rf)/100
    
    return

GetData(['SPY','KO','TSLA'],'1-1-2021','1-1-2022')

In [4]:
def statistics(x,y):
    global KO_vol
    
    print('''
    First we'll test if the two seperate distributions of sample returns came from normal 
    distributions. This is done using the normaltest() function from the SciPy Stats library, 
    which implements a D’Agostino and Pearson’s test of normality. This test takes advantage
    of the fact that individually, the test statistics for skewness and kurtosis are normally
    distributed. Therefore, the sum of their squares has a chi-squared distribution with 2
    degrees of freedom, which provides one option to test for normality.
    ''')
    
    k1,p1=sp.stats.normaltest(x)
    k2,p2=sp.stats.normaltest(y)
    
    print("KO p-value:"," ",round(p1,7))
    print("TSLA p-value:",round(p2,7))
    
    print('''
    With each test having p-values<.01, we can safely reject the null hypothesis that these 
    samples came from normally distributed populations at any reasonable confidence level.
    While the D’Agostino and Pearson’s test only tests for normality with regards to skew and 
    kurtosis, the null hypothesis was rejected with high confidence so it's safe to say the 
    population distributions are not normaly distributed. The Levene test can now be used via 
    the levene() function from the SciPy Stats library to determine if there is a statistically 
    significant difference between population variances. 
    
    A Levene test is more robust than a standard F-test, because it creates a test statistic
    using transformed data rather than the raw data entered. This transformed data is the 
    absolute value of the deviation(difference) between each data point and the mean of the ith 
    subgroup. In our case the transformed data is the deviation between each data point and the 
    median of the ith subgroup making it a Brown–Forsythe test. From the previous test we know 
    our the population distributions are likely skewed, so using the Brown–Forsythe will help 
    compensate for this.
    ''')
    
    l,L_p=sp.stats.levene(x,y, center = "median")
    
    print("Levene p-value:"," ",L_p)
    
    print('''
    This extreme p-value rejects the null hypothesis that the variances of the two population 
    distributions are equal at any reasonable confidence level. With the formula for annualized
    volatility being annualized standard deviation of returns, we can determine that because
    the variances for KO and TSLA are statistically different, so are their volatilities. 
    ''')

    print('''
    Now that we have determined that the difference in the varainces of the two samples is
    statistically significant, we can test to see if there is a statisically significant 
    difference between the population means. This is done with the ttest_ind() function from the 
    SciPy Stats library, which is a students t-test that can be designated to assume unequal variances, 
    making it a Welch's t-test. 
    ''')
    
    W,W_p=sp.stats.ttest_ind(x,y, equal_var = False )
    
    print("Welch's p-value:"," ",round(W_p,6))
    
    print('''
    The large p-value determines that the null hypothesis is accepted, meaning the 
    two seprate samples come from populations with equal means. 
    
    This is obviously counter-intuitive as the populations both are non-normal and have different 
    variances, so them sharing a mean doesn't make much sense. In the context of returns this outcome
    starts to make more sense. A majority of distributions of stock returns could be described
    as a skewed distribtuion with a mean about zero, with both the skew and mean being biased
    towards the general trend of the stock. So it make sense that both KO and TSLA would share a mean
    around zero, each scaled and skewed in regards to their overall performance. 
    
    Due to both this result and the result of our D’Agostino and Pearson’s test, there may be concerns 
    regarding lack of normality. Normality is a condition to perform a t test, but with the two samples 
    being very large the test remains robust due to the CLT. To confirm we can look at the results from 
    a Wilcoxon test, another test between means which accounts for normality. The results for the Wilcoxon
    test show the p-values for the Welch's and Wilcoxon test are similar, and confirms that we accept the 
    null hypothesis.
    ''')

    W2,W_p2=sp.stats.wilcoxon(x,y)
    
    print("Wilcoxon p-value:"," ",round(W_p2,6))
    
    print('''
    The methods of testing used for this analysis, while thorough, are somewhat excessive considering
    we are only dealing with two very large samples. That being said the tests conducted provided 
    some interesting insight into details of the population distributions, as well as generally being
    more robust. 
    ''')

    return
    
statistics(data["KO"],data["TSLA"])


    First we'll test if the two seperate distributions of sample returns came from normal 
    distributions. This is done using the normaltest() function from the SciPy Stats library, 
    which implements a D’Agostino and Pearson’s test of normality. This test takes advantage
    of the fact that individually, the test statistics for skewness and kurtosis are normally
    distributed. Therefore, the sum of their squares has a chi-squared distribution with 2
    degrees of freedom, which provides one option to test for normality.
    
KO p-value:   0.0001789
TSLA p-value: 1e-07

    With each test having p-values<.01, we can safely reject the null hypothesis that these 
    samples came from normally distributed populations at any reasonable confidence level.
    While the D’Agostino and Pearson’s test only tests for normality with regards to skew and 
    kurtosis, the null hypothesis was rejected with high confidence so it's safe to say the 
    population distributions are not nor

In [4]:
def metrics(x,rf):
    
    print('''
    Annualized volatiliy is calculated using the simple formula of the standard deviation of the returns 
    multiplied by the square root of the number of trading days. 
    ''')
    
    KO_vol=np.std(x)*np.sqrt(252)
    print('Volatility of KO:', str(round(KO_vol*100,2))+"%")
    
    print('''
    
    
    95% value at risk is calculated both historically and via simulation. Simulation isn't 
    the most robust method, it is parametric and since we determined that the population 
    distribution is non-normal without figuring out it's specific distribution, using parametric
    methods for estimating value at risk is somewhat illogical. It's remains to show how the CLT
    still allows the simulation to make a reasonable estimation when compared to the 
    historical(non-parametric) statistics. 
    ''')
    sim_returns = np.random.normal(np.mean(x), np.std(x), 1000000)
    
    H_VAR_95 = np.percentile(x, 5)*np.sqrt(252)
    S_VAR_95 = np.percentile(sim_returns, 5)*np.sqrt(252)
    
    print('Historical 95% Value at Risk:', str(round(H_VAR_95*100,2))+"%")
    print('Simulated 95% Value at Risk:', str(round(S_VAR_95*100,2))+"%")
    
    print('''
    This means that we can be 95% confident that over the next year, KO wont suffer losses worse
    than''', H_VAR_95)
    
    print('''
    
    
    Sharpe ratio is calulated with the standard formula of the mean of returns devided by the 
    standard deviation of returns. Included is both the Sharpe Ratio for returns and excess returns.
    For the risk-free rate I'm using the average yield of the 5-year treasury bond throughout 2021. 
    ''')
    
    sharpe_KO=np.mean(x)/(np.std(x))
    sharpe_KO2=(np.mean(x)-rf)/np.std(x)
    
    print('Sharpe Ratio:', sharpe_KO)
    print('Sharpe Ratio(Excess Returns):', sharpe_KO2)
    
    print('''
    With both Sharpe Ratios being less than one, neither provides an optimistic result for the future
    returns of KO.
    ''')
   
    print('''  
    
    Dowside deviation is calulated by first isolating all returns less than the minimum acceptable
    return(MAR), which in our case is the risk-free rate discussed earlier. The root sum squared of
    these isolated returns is calculated to yield the daily dowside deviation, which when multiplied
    by the square root of 252 yields the annualized downside deviation. 
    ''')

    sub_MAR_ret=(x-rf)[(x-rf)<0]
    Down_dev=np.sqrt(sum((sub_MAR_ret)**2)/(len(sub_MAR_ret)))*np.sqrt(252)

    print('Downside Deviation:', str(round(Down_dev*100,2))+"%")
    
    print('''
    
    
    Maximum drawdown is the trickiest metric to find in terms of data manipulation. First the daily
    returns for KO are converted into daily cumulative returns. The function cummax() is then used, 
    which returns the maximum cumulative return prior to and including the given day. The ratio
    of maximum cumulative returns over current cumulative returns yields the current drawdown(if there
    is one). The maximum of these ratios is the maximum drawdown for the year 2021.
    ''')
    cum_ret = np.cumsum(x)
    DD = (1 + cum_ret.cummax())/(1 + cum_ret) - 1
    print('Maximum Drawdown:', (str(-round(max(DD)*100,2))+"%"))
    
    return

metrics(data["KO"],E_rf)


    Annualized volatiliy is calculated using the simple formula of the standard deviation of the returns 
    multiplied by the square root of the number of trading days. 
    
Volatility of KO: 14.75%

    
    
    95% value at risk is calculated both historically and via simulation. Simulation isn't 
    the most robust method, it is parametric and since we determined that the population 
    distribution is non-normal without figuring out it's specific distribution, using parametric
    methods for estimating value at risk is somewhat illogical. It's remains to show how the CLT
    still allows the simulation to make a reasonable estimation when compared to the 
    historical(non-parametric) statistics. 
    
Historical 95% Value at Risk: -24.41%
Simulated 95% Value at Risk: -23.32%

    This means that we can be 95% confident that over the next year, KO wont suffer losses worse
    than -0.24414124450767702

    
    
    Sharpe ratio is calulated with the standard formula of th

In [5]:
def CAPM(x,y,rf):
    print('''
    First the cumulative returns for 2021 are calculated for both SPY and TSLA, these cumulative returns
    will serve as our "benmark" returns.  
    
    I calculated beta two methods which yielded similar results. The first is using a linear regression 
    model, where TSLA returns are modeled as a function of market(SPY) returns. The coefficient for SPY
    would then be our beta for the CAPM model. The Second method was using the covariance/variance method,
    where the covariance of returns is divided by the variance of SPY to get beta. The formulas for alpha
    remain the same but use the two seperately calculated betas.
    ''')
    bench_mkt = np.prod(x+1)-1
    bench_ret = np.prod(y+1)-1
    
    #Beta calculated via covariance/varaince method
    beta2=np.cov(x,y)[1,0]/np.var(x)
    #Beta calculated via linear regression
    beta = sp.stats.linregress(x, y= y)[0]
    #Alpha calculated via covariance/varaince method
    alpha2 = bench_ret-(rf+(beta2*(bench_mkt-rf)))
    #Alpha calculated via linear regression
    alpha = bench_ret-(rf+(beta*(bench_mkt-rf)))
    

    print('Beta(regression):', round(beta2,3))
    print('Alpha(regression):', round(alpha2,3))
    print()
    print('Beta(cov):', round(beta,3))
    print('Alpha(cov):', round(alpha,3))
    
    print('''
    The results from the two methods are very similar. Both calculations for beta show that TSLA is nearly
    twice as volatile as the benchmark. This helps explain our low value for alpha. 
    
    Alpha is the difference between TSLA returns for 2021 and the expected market returns assuming the 
    market shares the same volatility as TSLA. With alpha being negative, it shows that if the market
    was as volatile as TSLA it would likely outperform it, so the volatility of TSLA isn't desirable for
    investors.
    ''')
    return
    
CAPM(data["SPY"],data["TSLA"],E_rf)


    First the cumulative returns for 2021 are calculated for both SPY and TSLA, these cumulative returns
    will serve as our "benmark" returns.  
    
    I calculated beta two methods which yielded similar results. The first is using a linear regression 
    model, where TSLA returns are modeled as a function of market(SPY) returns. The coefficient for SPY
    would then be our beta for the CAPM model. The Second method was using the covariance/variance method,
    where the covariance of returns is divided by the variance of SPY to get beta. The formulas for alpha
    remain the same but use the two seperately calculated betas.
    
Beta(regression): 1.955
Alpha(regression): -0.314

Beta(cov): 1.948
Alpha(cov): -0.312

    The results from the two methods are very similar. Both calculations for beta show that TSLA is nearly
    twice as volatile as the benchmark. This helps explain our low value for alpha. 
    
    Alpha is the difference between TSLA returns for 2021 and the exp